In [1]:
cd scripts/

/home/ubuntu/mount_point/facebook/scripts


In [26]:
import grid_generation as grid
# import feature_engineering as feat
import numpy as np
import xgboost_extra_features_model as xgbm1
from sklearn.preprocessing import LabelEncoder

g = grid.Grid(200, 50, 20, 5, pref = 'grid')
submission_name = 'grid_200_50_20_5_def_params_th7_n100'
g.generateCardinalityMatrix()

In [35]:



train11 = np.loadtxt(g.getGridFile(1 ,1), delimiter = ',', dtype = int, usecols = (0, 5))
top_t11 = np.loadtxt(g.getTopPlacesFolder(submission_name) + 'top_t_preds_1_1.csv', delimiter = ',', dtype = int)

y = np.zeros((len(top_t11)))

y1 = (train11[:,1]) == (top_t11[:, 1])
y2 = (train11[:,1]) == (top_t11[:, 2])
y3 = (train11[:,1]) == (top_t11[:, 3])

#y[y1] = 1
y[y2] = 1
#y[y3] = 3

# scale_pos_weight = 

data = np.loadtxt(g.getGridFile(1, 1), dtype = float, delimiter = ',')
data_feature = np.loadtxt(g.getFeaturesFile(submission_name, 1, 1),\
        delimiter = ',')
data_combined = np.hstack((data[:, :-1], data_feature[:, 1:], data[:, -1].reshape(-1, 1)))


mask = np.array(map(lambda x:g.M[1][1][x] > 7, data_combined[:, -1]))
masked_data = data_combined[mask, :]
X, x_transformer = xgbm1.trans_x(masked_data[:, 1:-1])
# Ytr, enc = xgbm1.trans_y(y[mask])

Ytr = y[mask]
import xgboost as xgb
dtrain = xgb.DMatrix(X, label = Ytr)

In [74]:
scale = float(np.sum(Ytr==1))/np.sum(Ytr==0)

In [81]:



xgb_params = {
            'objective': 'binary:logistic',
            'eta': 0.1,
            'max_depth': 3,
            'min_child_weight': 6,
            'gamma': 0.1,
            'subsample': 0.9,
            'colsample_bytree': 0.6,
            
            'nthread': 4,
            'silent': 1,
            
            'max_delta_step': 7
}
bst = xgb.train(xgb_params, dtrain, 25)
yproba = bst.predict(dtrain)
#preds = enc['encoder'].inverse_transform(np.argsort(yproba, axis = 1)[:, ::-1][:, :1])

In [8]:
len(g.M[1][1])

213

In [82]:
yproba[:20]

array([ 0.15584189,  0.17553727,  0.16703577,  0.15482272,  0.15507543,
        0.12036553,  0.15758735,  0.14806916,  0.14809692,  0.13034661,
        0.15320468,  0.13211037,  0.16591415,  0.18334128,  0.17261617,
        0.15873644,  0.1525169 ,  0.16070642,  0.17672908,  0.14319062], dtype=float32)

In [83]:
Ytr[:20]

array([ 0.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [88]:
y_preds = np.zeros((len(yproba), ))
y_preds[yproba > 0.23] = 1
tp = np.sum((Ytr == 1) & (y_preds == 1))
fp = np.sum((Ytr == 0) & (y_preds == 1))
print tp
print fp

6
8


In [89]:
def create_time_dict(t_cuts, time_factor, time_aug):
    
    t_slice = 24 / t_cuts
    time_dict = dict()
    for t in range(t_cuts):
        
        t_min = 2 * np.pi * (t * t_slice * 12 / 288)
        t_max = 2 * np.pi * (((t + 1) * t_slice * 12 - 1) / 288)
        sin_t_start = np.round(np.sin(t_min)+1, 4) * time_factor
        sin_t_stop = np.round(np.sin(t_max)+1, 4) * time_factor
        cos_t_start = np.round(np.cos(t_min)+1, 4) * time_factor
        cos_t_stop = np.round(np.cos(t_max)+1, 4) * time_factor
        #print(t, (sin_t_start, sin_t_stop, cos_t_start, cos_t_stop))
        sin_t_min = min((sin_t_start, sin_t_stop))
        sin_t_max = max((sin_t_start, sin_t_stop))
        cos_t_min = min((cos_t_start, cos_t_stop))
        cos_t_max = max((cos_t_start, cos_t_stop))

        time_dict[t] = [sin_t_min, sin_t_max, cos_t_min, cos_t_max]
        t_min = 2 * np.pi * ((t * t_slice - time_aug) * 12 / 288)
        t_max = 2 * np.pi * ((((t + 1) * t_slice + time_aug)* 12 - 1) / 288)
        sin_t_start = np.round(np.sin(t_min)+1, 4) * time_factor
        sin_t_stop = np.round(np.sin(t_max)+1, 4) * time_factor
        cos_t_start = np.round(np.cos(t_min)+1, 4) * time_factor
        cos_t_stop = np.round(np.cos(t_max)+1, 4) * time_factor
        sin_t_min = min((sin_t_start, sin_t_stop, sin_t_min))
        sin_t_max = max((sin_t_start, sin_t_stop, sin_t_max))
        cos_t_min = min((cos_t_start, cos_t_stop, cos_t_min))
        cos_t_max = max((cos_t_start, cos_t_stop, cos_t_max))
        time_dict[t] += [sin_t_min, sin_t_max, cos_t_min, cos_t_max]
        
    return time_dict


In [105]:
t_d = create_time_dict(4, np.max(m)/2, 2)

In [106]:
t_d

{0: [3.1306843457648288,
  3.1306843457648288,
  6.2613686915296576,
  6.2613686915296576,
  3.1306843457648288,
  3.1306843457648288,
  6.2613686915296576,
  6.2613686915296576],
 1: [3.1306843457648288,
  3.1306843457648288,
  6.2613686915296576,
  6.2613686915296576,
  3.1306843457648288,
  3.1306843457648288,
  6.2613686915296576,
  6.2613686915296576],
 2: [3.1306843457648288,
  3.1306843457648288,
  6.2613686915296576,
  6.2613686915296576,
  3.1306843457648288,
  3.1306843457648288,
  6.2613686915296576,
  6.2613686915296576],
 3: [3.1306843457648288,
  3.1306843457648288,
  6.2613686915296576,
  6.2613686915296576,
  3.1306843457648288,
  3.1306843457648288,
  6.2613686915296576,
  6.2613686915296576]}

In [102]:
m = minute = 2*np.pi*((data11[:, 4]//5)%288)/288

In [101]:
data11 = np.loadtxt(g.getGridFile(1 ,1), delimiter = ',')

In [103]:
m

array([ 4.14515697,  6.06501915,  4.25424005, ...,  0.78539816,
        0.56723201,  4.75602221])

In [104]:
np.max(m)

6.2613686915296576